This program allows exploration of the Eiden Lab microarray data sets 2019.03, 2019.04, and 2019.12. Press the double arrow above and then confirm "restart and run all cells"

In [ ]:
import matplotlib
import pandas as pd
import ipywidgets as widgets
import io
import plotly.express as px
import plotly.io as pio
pio.templates.default = "simple_white"
from IPython.display import display
from ipywidgets import interact_manual, interactive,GridspecLayout, AppLayout, Button, Layout, Widget

In [ ]:
##DATA FOR STRESS COMPARISONS
#retreive files with ANOVA data
a0 = pd.read_csv("results_part1.csv")
b0 = pd.read_csv("results_part2.csv")
c0 = pd.read_csv("results_part3.csv")
#d0 = pd.read_csv("results_part4.csv")

#combine them
result0 = pd.concat([a0,b0,c0])
result0.dropna(subset = ["Gene Symbol"], inplace=True)
result = result0.groupby(['Gene Symbol','Tissue','Comparison',"Experiment"])[['p-value','Fold-change']].agg([('mean')]).reset_index()
result.columns = result.columns.get_level_values(0)


#load the expression data and combine/ fix older ones
a=pd.read_csv("Expressions_all_part1.csv", dtype={'Gene Symbol': object, 'Experiment':object})
b=pd.read_csv("Expressions_all_part2.csv", dtype={'Gene Symbol': object, 'Experiment':object})
c=pd.read_csv("Expressions_all_part3.csv", dtype={'Gene Symbol': object, 'Experiment':object})
d=pd.read_csv("Expressions_all_part4.csv", dtype={'Gene Symbol': object, 'Experiment':object})
e=pd.read_csv("Expressions_all_part5.csv", dtype={'Gene Symbol': object, 'Experiment':object})
f=pd.read_csv("Expressions_all_part6.csv", dtype={'Gene Symbol': object, 'Experiment':object})
g=pd.read_csv("Expressions_all_part7.csv", dtype={'Gene Symbol': object, 'Experiment':object})
h = pd.read_csv("Expressions_all_part8.csv", dtype={'Gene Symbol': object, 'Experiment':object})
i = pd.read_csv("Expressions_all_part9.csv", dtype={'Gene Symbol': object, 'Experiment':object})
j = pd.read_csv("Expressions_all_part10.csv", dtype={'Gene Symbol': object, 'Experiment':object})
k = pd.read_csv("Expressions_all_part11.csv", dtype={'Gene Symbol': object, 'Experiment':object})
l = pd.read_csv("Expressions_all_part12.csv", dtype={'Gene Symbol': object, 'Experiment':object})
#combine them
expressions = pd.concat([a,b,c,d,e,f,g, h,i,j,k,l], sort=False).reset_index()
#clean up a bit
expressions.drop(expressions.columns[0], axis=1, inplace=True)
expressions.drop(expressions.columns[0], axis=1, inplace=True)

In [ ]:
from traitlets import traitlets
##Create a class of buttons which allows me to check which one was pressed
class LoadedButton(widgets.Button):
    """A button that can hold a value as a attribute."""

    def __init__(self, value=None, *args, **kwargs):
        super(LoadedButton, self).__init__(*args, **kwargs)
        # Create the value attribute.
        self.add_traits(value=traitlets.Any(value))


button_layout = widgets.Layout(border='solid 1px black', height='auto', width='auto')
def create_expanded_button(description, button_style):
    return LoadedButton(description=description, button_style=button_style, layout=button_layout)


In [ ]:
##define GUI  
def choose_gene(change):
    if dropdown.value not in genes2:
        if dropdown.value != '':
            genes2.append(dropdown.value)
            with outgenes:
                display(dropdown.value)
                
graph_out1 = widgets.Output()  
graph_out2 = widgets.Output()
 
def on_value_change(change):
    #just be here
    print('')

            
def on_reset_button_clicked(reset_button):
    fig.data = []
    th_go_button.button_style = ''
    th_go_button.value == "no threshold"
    no_thresh_button.button_style = ''
    no_thresh_button.value = 'threshold'
    graph_out1.clear_output() 
    graph_out2.clear_output()
    outerrors.clear_output()
    outgenes.clear_output()
    outgraph.clear_output()
    genes2 = []
    mouse_means2, stats, sig_genes = [],[],[]
    
def on_go_button_clicked(go_button):
    mouse_means2, stats, sig_genes = get_vals(go_button)
#Mark errors
    if len(genes2) < 1:
        with outerrors:
            print("Choose at least one Gene")
    else:
        #print(mouse_means2)
       ###Graph expression   
        import plotly.graph_objects as go
        with graph_out1:
            fig =px.bar(
                mouse_means2, x="Geno and Condition", 
                y="average",
                facet_col = "Gene Symbol",
                color = "Experiment",
                barmode = 'group',
                color_discrete_sequence= px.colors.sequential.Plasma,
                error_y="error"
            ).for_each_trace(lambda t: t.update(name=t.name.replace("=",": ")))
            for i, a in enumerate(fig['layout']['annotations']):
                    if a['text'][:5]=='Exper':
                           a['text'] = ''
                    if a['text'][:5]=='Gene ':
                            a['text'] = a['text'].split("=")[1]
            fig.update_layout(
                title="Expression levels of selected genes",
                yaxis_title="Expression",
                xaxis={'categoryorder':'total descending'})
            fig.update_xaxes(automargin=True,tickangle=45)
            fig.show()

       ###Graph statistics
        with graph_out2:
            fig2 = px.bar(stats, x="Comparison", y="Fold-change",
                                #width=800, height=400,
                                facet_col = "Gene Symbol",
                                facet_row = "Experiment",
                                color_discrete_sequence= px.colors.sequential.Plasma,
                                #color = "p-value", range_color=[0,p_threshold.value], #Change so user input tells it what p and fc to use
                                color = "Experiment",
                                hover_name = "Tissue",
                                hover_data = ["Experiment", "Fold-change", "p-value"]
                          ).for_each_trace(lambda t: t.update(name=t.name.replace("=",": ")))
            fig2.update_yaxes(title_text='')
            fig2.update_xaxes(title_text='', tickangle=45)
            for i, a in enumerate(fig2['layout']['annotations']):
                if a['text'][:5]=='Exper':
                        a['text'] = ''
                if a['text'][:5]=='Gene ':
                        a['text'] = a['text'].split("=")[1]
            fig2.update_layout(title="ANOVA results",
                               margin=dict(

                                b=250))
            #fig.update_xaxes(automargin=True)
            fig2.show()

def on_thresh_button_clicked(th_go_button):
    th_go_button.button_style = 'success'
    th_go_button.value = "threshold"

    #first clear any existing stuff out
    graph_out1.clear_output() 
    graph_out2.clear_output()
    outerrors.clear_output()
    outgenes.clear_output()
    outgraph.clear_output()
    genes2=[]
    

    mouse_means1 = expressions[expressions["Gene Symbol"].isin(genes2)]
    mouse_means2 = (mouse_means1.groupby(['Gene Symbol','Tissue','Geno and Condition',"Experiment"])['Expression']
         .agg([('average','mean'),('error','sem')])
         .reset_index())
    
    
    results_stringent = result[result["p-value"] <= p_threshold.value]#Change so user input tells it what p and fc to use
    results_stringent2=results_stringent[(results_stringent['Fold-change']>= fc_threshold.value) | (results_stringent['Fold-change']<= -fc_threshold.value)]
    a = results_stringent2[results_stringent2["Comparison"].isin(list(comps.value))]
    stats = a[a["Gene Symbol"].isin(genes2)]
    
    sig_genes0 = results_stringent2["Gene Symbol"].tolist()
    sig_genes = list(set(sig_genes0))
    
    
    #create next part of app
    dropdown.options = sig_genes
    display(app)
    return(mouse_means2, stats)

def no_thresh_button_clicked(no_thresh_button):
    no_thresh_button.button_style = 'success'
    no_thresh_button.value = "no threshold"

    #first clear any existing stuff out
    graph_out1.clear_output() 
    graph_out2.clear_output()
    outerrors.clear_output()
    outgenes.clear_output()
    outgraph.clear_output()
    genes2=[]
    sig_genes0 = list(result["Gene Symbol"])
    sig_genes = list(set(sig_genes0))
    mouse_means1 = expressions[expressions["Gene Symbol"].isin(genes2)]
    mouse_means2 = (mouse_means1.groupby(['Gene Symbol','Tissue','Geno and Condition',"Experiment"])['Expression']
            .agg([('average','mean'),('error','sem')])
            .reset_index())
    
    
    a = result[result["Comparison"].isin(list(comps.value))]
    stats = a[a["Gene Symbol"].isin(genes2)]
    #create next part of app
    dropdown.options = sig_genes
    display(app)
    return(mouse_means2, stats)


def get_vals(go_button):

    if th_go_button.value == "threshold":

        a = result[result["Comparison"].isin(list(comps.value))]
        stats = a[a["Gene Symbol"].isin(genes2)]
        sig_genes = result["Gene Symbol"].tolist()
        mouse_means1 = expressions[expressions["Gene Symbol"].isin(genes2)]
        mouse_means2 = (mouse_means1.groupby(['Gene Symbol','Tissue','Geno and Condition',"Experiment"])['Expression']
             .agg([('average','mean'),('error','sem')])
             .reset_index())

        
    if th_go_button.value == "no threshold":
        
        a = result[result["Comparison"].isin(list(comps.value))]
        stats = a[a["Gene Symbol"].isin(genes2)]
        
        sig_genes0 = result["Gene Symbol"].tolist()
        sig_genes = list(set(sig_genes0))
        mouse_means1 = expressions[expressions["Gene Symbol"].isin(genes2)]
        mouse_means2 = (mouse_means1.groupby(['Gene Symbol','Tissue','Geno and Condition',"Experiment"])['Expression']
             .agg([('average','mean'),('error','sem')])
             .reset_index())

    return(mouse_means2, stats, sig_genes)


In [ ]:
allgenes = list(set(result["Gene Symbol"]))
style = {'description_width': 'initial'}
genes2 = []             
spacer = widgets.Label('')
genetitle = widgets.Label('Select Genes')

dropdown =  widgets.Combobox(
    options= ["allgenes"],
    description='Gene',
    placeholder='Type here',
    layout={'width': '80%'})

   
outgenes = widgets.Output()   
outgraph = widgets.Output(layout={'border': '1px solid black'})  

        
gene_button = widgets.Button(description = 'Enter Gene', 
                button_style = '', 
                display='flex',
                flex_flow='column',
                align_items='stretch')
gene_button.on_click(choose_gene)

go_button = widgets.Button(description = 'Produce Output', 
                           button_style = '', 
                            display='flex',
                            flex_flow='column',
                            align_items='stretch')
go_button.on_click(on_go_button_clicked)

reset_button = widgets.Button(description = 'Reset App', 
                           button_style = '', 
                            display='flex',
                            flex_flow='column',
                            align_items='stretch')
reset_button.on_click(on_reset_button_clicked)


gene_button.observe(dropdown.value)
go_button.observe(dropdown.value)

p_threshold = widgets.BoundedFloatText(
    value=1,
    description='p-value:',
    disabled=False,
    width = '80%',
    min=0,
    max=1,
    step=0.01,
    style=style)
p_threshold.observe(on_value_change, names='value')

fc_threshold = widgets.BoundedFloatText(
    value=2,
    description='Fold-change (+/-):',
    disabled=False,
    min=0,
    max=50,
    step=0.5,
    width = '80%',
    style=style)
fc_threshold.observe(on_value_change, names='value')


graph_out1 = widgets.Output()  
graph_out2 = widgets.Output() 

In [ ]:
comps = widgets.SelectMultiple(
    options=list(set(result["Comparison"])),
    layout=Layout(width='100%', height = '230px'),
    #rows=10,
    description='Compare: ',
    disabled=False
)


In [ ]:
threshold_title = widgets.Label('First, set p and fold-change')
th_go_button = LoadedButton(description = 'Set Thresholds',
                           button_style = '', 
                            display='flex',
                            flex_flow='column',
                            align_items='stretch',
                            value="no threshold",
                            layout=Layout(width='90%'))
th_go_button.on_click(on_thresh_button_clicked)


no_thresh_button = LoadedButton(description = 'No Thresholds', 
                           button_style = '', 
                            display='flex',
                            flex_flow='column',
                            align_items='stretch',
                            value = "threshold",
                            layout=Layout(height='80px'))
no_thresh_button.on_click(no_thresh_button_clicked)

genes_out_title = widgets.Label('Genes Chosen:')


genesbox = widgets.VBox([genetitle,dropdown])
genes_out = widgets.VBox([spacer,genes_out_title, outgenes])
buttons = widgets.HBox([gene_button, go_button, reset_button])
#cellsbox = widgets.VBox([celllabel, Cells_drop])
genechoices = widgets.HBox([genesbox, genes_out])

outerrors = widgets.Output() 
no_thresh = widgets.VBox([spacer, no_thresh_button])
graphit = widgets.VBox([graph_out1, graph_out2])
top_app = widgets.VBox([genechoices, outerrors, buttons]) ##Put seletcors for p and fc values here
app = widgets.VBox([top_app, graphit])
thresholds0 = widgets.VBox([threshold_title,p_threshold, fc_threshold,comps, th_go_button])
thresholds=widgets.HBox([thresholds0, no_thresh])

thresholds